In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = "facebook/bart-base"  # or "facebook/bart-large"

tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2_Train.csv")
dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/Train/Train_Anticipation_All.csv")
test_dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2/MViTv2_Test_Anticipation.csv")


In [ ]:
def preprocess_data(examples):
    q = "What is the next action?"
    inputs = [f"Question: {q} Context: {c}" for c in zip(examples["context"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=25)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched = True)
print(dataset)


In [ ]:
num_epochs = 1 
from transformers import TrainingArguments, Trainer
for i in range(num_epochs):

    training_args = TrainingArguments(
        output_dir="./bart_qa",
        evaluation_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=30,
        weight_decay=0.01,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset = test_dataset["train"],
        # predict_with_generate = True,
        # preprocess_logits_for_metrics=preprocess_logits_for_metrics
        # compute_loss_func=compute_loss_func
    )

    trainer.train()


In [ ]:
model.save_pretrained("./bart_anticipation", from_pt=True) 
tokenizer.save_pretrained("./bart_anticipation")

In [ ]:
# from transformers import EncoderDecoderModel, BertTokenizer

# # Load trained model and tokenizer
model_name = "./bart_anticipation"  # Change to your actual model path
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to("cuda")

# Ensure correct config

def generate_answer(question, context):
    # Format input as: "question: ... context: ..."
    input_text = f"question: {question}  context: {context}"

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to("cuda")

    # Generate response
    output_ids = model.generate(inputs.input_ids, max_length=100, num_beams=5)

    # Decode output tokens
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# print(generate_answer("What is the next action?", "palpate landmark, take swab, prep site"))

In [ ]:
test_dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2/MViTv2_Test_Anticipation.csv")
test_dataset = test_dataset["train"]
# questions = test_dataset["question"]
context = test_dataset["context"]
labels = test_dataset["answer"]    
print(test_dataset)
print(labels)
print(context)

In [ ]:
generated_answers = []
for i in range(len(context)):
    # print(questions[i], context[i])
    generated_answers.append(generate_answer("What is the next action?", context[i]))
    print(generated_answers[i], labels[i])
print(generated_answers)
print(labels)

In [ ]:
correct = 0
total = 0
for i in range(len(generated_answers)):
    if((generated_answers[i])==(labels[i])):
        correct = correct+1
        print(correct, total)
    total = total+1
print(correct/total)